# Getting Started with Geospatial Studio

This notebook walks you through the essential workflows for using the Geospatial Studio:

1. **API Setup** - Configure your environment
2. **Onboard Sandbox Models** - Set up placeholder models
3. **Load Existing Inferences** - Import example outputs
4. **Run Inference** - Use pre-tuned models
5. **Fine-tune Models** - Train models on your data

## Prerequisites

- Geospatial Studio deployed and running
- Access to Studio UI at `https://localhost:4180`
- Python environment with required packages

---

## Step 1: API Key Setup

### Generate API Key

1. Navigate to [https://localhost:4180](https://localhost:4180)
2. Log in with default credentials:
   - Username: `testuser`
   - Password: `testpass123`
3. Click **"Manage your API keys"** link
4. Generate a new API key and copy it


### Configure Environment Variables

In [ ]:
# Set your API key (paste the key you generated)
import os

os.environ['STUDIO_API_KEY'] = "<paste your API key here>"
os.environ['UI_ROUTE_URL'] = "https://localhost:4180"

# Verify the variables are set
print(f"API Key: {os.environ['STUDIO_API_KEY'][:10]}...")
print(f"UI URL: {os.environ['UI_ROUTE_URL']}")

---

## Step 2: Onboard Sandbox Models

Sandbox models are placeholder pipelines useful for:
- Onboarding existing inferences
- Testing tuned models

In [ ]:
%%bash
./deployment-scripts/add-sandbox-models.sh

---

## Step 3: Onboard Existing Inference (Example Data)

Load a pre-computed inference to explore the platform.

This will:
1. Start a pipeline to pull the data
2. Set it up in the platform
3. Make it visible in the Inferences page

In [ ]:
%%bash
# Run the populate script and select "AGB Data - Karen, Nairobi, Kenya"
python populate-studio/populate-studio.py inferences

**After running the above:**
- Navigate to the **Inferences** page in the UI
- You should see the example you just added

---

## Step 4: Onboard a Pre-Tuned Model and Run Inference

### 4.1 Onboard Tuning Task Templates

Templates make it easier to configure tuning tasks.

In [ ]:
%%bash
# Select: 1. Segmentation - Generic template v1 and v2 models: Segmentation
python populate-studio/populate-studio.py templates

### 4.2 Onboard a Tuned Model

We'll onboard the `prithvi-eo-flood` model from a URL.

In [ ]:
%%bash
# Select: prithvi-eo-flood - prithvi-eo-flood
python populate-studio/populate-studio.py tunes

**Check the UI:**
- Navigate to **Models/Tunes** page
- Wait for the download to complete
- Copy the `tune_id` for the next step

### 4.3 Run Inference with the Tuned Model

Now we'll run inference over a specific spatial and temporal domain.

In [ ]:
import json
import requests
import os

# IMPORTANT: Replace with your tune_id from the UI
tune_id = "<paste tune_id here>"

# Define the inference payload
payload = {
    "model_display_name": "geofm-sandbox-models",
    "location": "Dakhin Petbaha, Raha, Nagaon, Assam, India",
    "description": "Flood Assam local with sentinel aws",
    "spatial_domain": {
        "bbox": [
            [92.703396, 26.247896, 92.748087, 26.267903]
        ],
        "urls": [],
        "tiles": [],
        "polygons": []
    },
    "temporal_domain": [
        "2024-07-25_2024-07-28"
    ]
}

# Submit the inference request
url = f"{os.environ['UI_ROUTE_URL']}/studio-gateway/v2/tunes/{tune_id}/try-out"
headers = {
    "Content-Type": "application/json",
    "X-API-Key": os.environ['STUDIO_API_KEY']
}

response = requests.post(url, json=payload, headers=headers, verify=False)
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")

**Monitor Progress:**
- Go to the **Inferences** page in the UI
- Watch the job progress
- Access results via [MinIO UI](https://localhost:9001)

---

## Step 5: Fine-Tune a Model from Your Dataset

### 5.1 Onboard a Tuning Dataset

In [ ]:
%%bash
# Select: "Wildfire burn scars"
python populate-studio/populate-studio.py datasets

**Check the UI:**
- Navigate to **Datasets** page
- Dataset will show as "Pending" initially
- Wait for it to complete (a few minutes)
- Copy the `dataset_id`

### 5.2 Onboard Backbone Model

In [ ]:
%%bash
# Select: "Prithvi_EO_V2_300M"
python populate-studio/populate-studio.py backbones

**Copy the `base_model_id` from the response or UI.**

### 5.3 Onboard Tuning Templates (if not done already)

In [ ]:
%%bash
# Select: 1. Segmentation - Generic template v1 and v2 models: Segmentation
python populate-studio/populate-studio.py templates

**Copy the `tune_template_id` from the response or UI.**

### 5.4 Submit Fine-Tuning Job

⚠️ **Note for Mac Users:** If you're on a Mac without NVIDIA GPUs, see the [Mac GPU Tuning](#mac-gpu-tuning) section below.

In [ ]:
import json
import requests
import os

# IMPORTANT: Replace these IDs with your actual IDs from previous steps
dataset_id = "<paste dataset_id here>"
base_model_id = "<paste base_model_id here>"
tune_template_id = "<paste tune_template_id here>"

# Define the tuning payload
payload = {
    "name": "burn-scars-demo",
    "description": "Segmentation",
    "dataset_id": dataset_id,
    "base_model_id": base_model_id,
    "tune_template_id": tune_template_id,
    "train_options": {
        "model_input_data_spec": [
            {
                "bands": [
                    {"index": "0", "band_name": "Blue", "scaling_factor": "0.0001", "RGB_band": "B"},
                    {"index": "1", "band_name": "Green", "scaling_factor": "0.0001", "RGB_band": "G"},
                    {"index": "2", "band_name": "Red", "scaling_factor": "0.0001", "RGB_band": "R"},
                    {"index": "3", "band_name": "NIR_Narrow", "scaling_factor": "0.0001"},
                    {"index": "4", "band_name": "SWIR1", "scaling_factor": "0.0001"},
                    {"index": "5", "band_name": "SWIR2", "scaling_factor": "0.0001"}
                ],
                "connector": "sentinelhub",
                "collection": "hls_l30",
                "file_suffix": "_merged.tif",
                "modality_tag": "HLS_L30"
            }
        ],
        "label_categories": [
            {"id": "-1", "name": "Ignore", "color": "#000000", "opacity": 0, "weight": None},
            {"id": "0", "name": "No data", "color": "#000000", "opacity": 0, "weight": None},
            {"id": "1", "name": "Fire Scar", "color": "#ab4f4f", "opacity": 1, "weight": None}
        ]
    },
    "model_parameters": {
        "data": {"check_stackability": "false"},
        "runner": {"max_epochs": "5"}
    }
}

# Submit the tuning job
url = f"{os.environ['UI_ROUTE_URL']}/studio-gateway/v2/submit-tune"
headers = {
    "Content-Type": "application/json",
    "X-API-Key": os.environ['STUDIO_API_KEY']
}

response = requests.post(url, json=payload, headers=headers, verify=False)
print(f"Status Code: {response.status_code}")
print(f"Response: {json.dumps(response.json(), indent=2)}")

# Save the tune_id for later
if response.status_code == 200:
    new_tune_id = response.json().get('tune_id')
    print(f"\n✅ Tuning job submitted! Tune ID: {new_tune_id}")

**Monitor Training:**
- Open [MLflow UI](https://localhost:5000)
- Track training progress and metrics
- Wait for training to complete (depends on epochs and data size)

### 5.5 Run Inference with Your Tuned Model

After training completes, run inference with your newly tuned model.

In [ ]:
import json
import requests
import os

# IMPORTANT: Replace with your new tune_id from the training job
tune_id = "<paste new tune_id here>"

# Expanded inference payload with full configuration
payload = {
    "model_display_name": "geofm-sandbox-models",
    "location": "Red Bluff, California, United States",
    "description": "Park Fire Aug 2024",
    "spatial_domain": {
        "bbox": [],
        "urls": [
            "https://geospatial-studio-example-data.s3.us-east.cloud-object-storage.appdomain.cloud/examples-for-inference/park_fire_scaled.tif"
        ],
        "tiles": [],
        "polygons": []
    },
    "temporal_domain": ["2024-08-12"],
    "pipeline_steps": [
        {"status": "READY", "process_id": "url-connector", "step_number": 0},
        {"status": "WAITING", "process_id": "terratorch-inference", "step_number": 1},
        {"status": "WAITING", "process_id": "postprocess-generic", "step_number": 2},
        {"status": "WAITING", "process_id": "push-to-geoserver", "step_number": 3}
    ],
    "post_processing": {
        "cloud_masking": "False",
        "ocean_masking": "False",
        "snow_ice_masking": None,
        "permanent_water_masking": "False"
    },
    "model_input_data_spec": [
        {
            "bands": [
                {"index": "0", "RGB_band": "B", "band_name": "Blue", "scaling_factor": "0.0001"},
                {"index": "1", "RGB_band": "G", "band_name": "Green", "scaling_factor": "0.0001"},
                {"index": "2", "RGB_band": "R", "band_name": "Red", "scaling_factor": "0.0001"},
                {"index": "3", "band_name": "NIR_Narrow", "scaling_factor": "0.0001"},
                {"index": "4", "band_name": "SWIR1", "scaling_factor": "0.0001"},
                {"index": "5", "band_name": "SWIR2", "scaling_factor": "0.0001"}
            ],
            "connector": "sentinelhub",
            "collection": "hls_l30",
            "file_suffix": "_merged.tif",
            "modality_tag": "HLS_L30"
        }
    ],
    "geoserver_push": [
        {
            "z_index": 0,
            "workspace": "geofm",
            "layer_name": "input_rgb",
            "display_name": "Input image (RGB)",
            "filepath_key": "model_input_original_image_rgb",
            "visible_by_default": "True",
            "geoserver_style": {
                "rgb": [
                    {"label": "RedChannel", "channel": 1, "maxValue": 255, "minValue": 0},
                    {"label": "GreenChannel", "channel": 2, "maxValue": 255, "minValue": 0},
                    {"label": "BlueChannel", "channel": 3, "maxValue": 255, "minValue": 0}
                ]
            }
        },
        {
            "z_index": 1,
            "workspace": "geofm",
            "layer_name": "pred",
            "display_name": "Model prediction",
            "filepath_key": "model_output_image",
            "visible_by_default": "True",
            "geoserver_style": {
                "segmentation": [
                    {"color": "#000000", "label": "ignore", "opacity": 0, "quantity": "-1"},
                    {"color": "#000000", "label": "no-data", "opacity": 0, "quantity": "0"},
                    {"color": "#ab4f4f", "label": "fire-scar", "opacity": 1, "quantity": "1"}
                ]
            }
        }
    ]
}

# Submit inference
url = f"{os.environ['UI_ROUTE_URL']}/studio-gateway/v2/tunes/{tune_id}/try-out"
headers = {
    "Content-Type": "application/json",
    "X-API-Key": os.environ['STUDIO_API_KEY']
}

response = requests.post(url, json=payload, headers=headers, verify=False)
print(f"Status Code: {response.status_code}")
print(f"Response: {json.dumps(response.json(), indent=2)}")

**View Results:**
- Navigate to **Inferences** page in UI
- Monitor job progress
- View results in GeoServer
- Access files via [MinIO UI](https://localhost:9001)

---

## Mac GPU Tuning (Alternative Workflow)

For Mac users without NVIDIA GPUs, you can prepare the tuning configuration in the Studio and run it locally with TerraTorch.

### Prepare Tuning Configuration

In [ ]:
import json
import requests
import os

# IMPORTANT: Replace these IDs with your actual IDs
dataset_id = "<paste dataset_id here>"
base_model_id = "<paste base_model_id here>"
tune_template_id = "<paste tune_template_id here>"

payload = {
    "name": "burn-scars-demo",
    "description": "Segmentation",
    "dataset_id": dataset_id,
    "base_model_id": base_model_id,
    "tune_template_id": tune_template_id,
    "model_parameters": {
        "runner": {"max_epochs": "10"},
        "optimizer": {"lr": 6e-05, "type": "AdamW"}
    }
}

# Get dry-run config
url = f"{os.environ['UI_ROUTE_URL']}/studio-gateway/v2/submit-tune/dry-run"
headers = {
    "Content-Type": "application/json",
    "X-API-Key": os.environ['STUDIO_API_KEY']
}

response = requests.post(url, json=payload, headers=headers, verify=False)

# Save config to file
with open('config.yaml', 'w') as f:
    f.write(response.text)

print("✅ Config saved to config.yaml")

### Localize Config Paths

In [ ]:
%%bash
./deployment-scripts/localize_config.sh config.yaml

### Run Training Locally

In [ ]:
%%bash
terratorch fit -c config.yaml

### Upload Tuned Model Back to Studio

After training completes, upload your checkpoint back to the Studio.Once its complete, you should see the it in the UI under the tunes/models page.

*(API endpoint for upload to be added)*
*(API call to try out inferenfe)*

---